1. Read mesh points for Blender (Phenotype)
2. Convert Blender xyz tensors to NN tensor format
3. Create NN input and output Targets
4. Initialize network model
5. Train 2D Ge input to NN model 3D (xyz) Ph output
6. Generate sample model output and blender formatted test data
7. Save 2D template model weights to file


In [2]:
import torch as torch
import torch.nn as nn

import numpy as np
import json 
import time
import NNArt as gy

import plotly as py
import plotly.offline as offline
import plotly.graph_objs as go

CURRENT_MODEL_SAVE_DIR = './save/'

offline.init_notebook_mode(connected=True) 
np.set_printoptions(precision=3, suppress=True)

In [3]:
import sys
sys.executable
from platform import python_version

print('CUDA is available: {} \n  on device: {} ({})\nfor pytorch: {} and python: {}\n  at: {}'.format(
    torch.cuda.is_available(),
    torch.cuda.current_device(), torch.cuda.get_device_name(torch.cuda.current_device()),
    torch.__version__,
    python_version(),
    sys.executable
    ))


CUDA is available: True 
  on device: 0 (GeForce GTX 1080 Ti)
for pytorch: 1.0.1 and python: 3.7.2
  at: D:\Anaconda3\envs\pytorch\python.exe


In [4]:
class Net(nn.Module):
    def __init__(self, layerSizes=[2,10,10,20]):
        super(Net, self).__init__()
        self.shape = layerSizes
        self.seq = nn.Sequential(
            nn.Linear(layerSizes[0],layerSizes[1]),
            nn.ELU(),          
            nn.Linear(layerSizes[1],layerSizes[2]),
            nn.ELU(),            
            nn.Linear(layerSizes[2],layerSizes[3]),
            nn.ELU()
        )
        
    def forward(self, x):
        return self.seq(x) 

In [5]:
def createRandomizedTargetsGP(g,p):
    '''
    Randomize mesh interior while maintaining shape and edge coordinates
    
    Defines an internal function createRandomizedInput(target, r0, r1) 
    that is run twice, once for g and once for p, using the same random numbers
    
    '''
    def createRandomizedInput(target, r0, r1):
        '''      
        Randomize mesh interior while maintaining shape and edge coordinates

        First randomizing morphology (requireds swapping time and morph axis)
        Then time axis

        Double first and last elements and 
        add a random fraction of the difference between rows to each row trimming the last row

        This will add one to the row and column count

        '''

        ##### Randomize Morphology #####
        #swap time and morph tensors
        timeSwap = np.swapaxes(target,0,1).data.numpy()
        #double first and last time morphs 
        timeSwapPrePost = np.concatenate((timeSwap[:1],timeSwap,timeSwap[-1:]),axis=0)

        #compute difference between time steps
        #should be all zeros except time in tensor dim 2
        timeDifs = timeSwapPrePost[:-1]-timeSwapPrePost[1:]
        #add random percent to diff and add to time morphs    
        timeSwapRandom = r0*timeDifs+timeSwapPrePost[1:]

        #unswap swaped tensors
        target2 =  np.swapaxes(timeSwapRandom,0,1)

        ##### Randomize Time #####
        timePrePost = np.concatenate((target2[0:1],target2,target2[-1:]),axis=0)
        timeDif = timePrePost[:-1]-timePrePost[1:]
        timeMorphRandom = r1*timeDif+timePrePost[1:]

        return timeMorphRandom

    r = np.random.rand(1)
    rr = np.random.rand(1)
    
    g = createRandomizedInput(g,r,rr)
    p = createRandomizedInput(p,r,rr)
    return torch.Tensor(g), torch.Tensor(p)


In [6]:
def phTrain(phModel, parms):
    epochs = parms['epochs']
    lRate = parms['lRate']
    numBatch = parms['numBatch']
    
    minLossP = 10
    maxLossP = 0
    phOptimizer = torch.optim.Adam(phModel.parameters(), lr=lRate)
    for epoch in range(epochs):
        relMapIn, phTargetOut = createRandomizedTargetsGP(parms['relMap'],parms['phTarget'])
        for b in range(numBatch):
            phOptimizer.zero_grad()
            phOutput = phModel(relMapIn[b])
            lossP = criterion(phOutput, phTargetOut[b])
            (lossP).backward()
            phOptimizer.step()
            minLossP = min(lossP.data.item(), minLossP)
            maxLossP = max(lossP.data.item(), maxLossP)
            
        if (epoch) % (epochs/10) == 0:
            print("Epoch {0:,.0f} - loss: {1:,.5f}".format(epoch, lossP.data.item()))
    print("Epoch {0:,.0f} - loss: {1:,.5f}:{2:,.5f}".format(epoch+1, minLossP, maxLossP))
    return

In [7]:
''' 
1. read mesh points for blender (Phenotype)

Run multiple times with different file prefix to generate and save multiple models

'''
pre = 'PetalS0T'
with open(CURRENT_MODEL_SAVE_DIR+pre+'0.json', 'r') as fp:
    bc0 = np.array(json.load(fp))

with open(CURRENT_MODEL_SAVE_DIR+pre+'1.json', 'r') as fp:
    bc1 = np.array(json.load(fp))

with open(CURRENT_MODEL_SAVE_DIR+pre+'2.json', 'r') as fp:
    bc2 = np.array(json.load(fp))

with open(CURRENT_MODEL_SAVE_DIR+pre+'3.json', 'r') as fp:
    bc3 = np.array(json.load(fp))

with open(CURRENT_MODEL_SAVE_DIR+pre+'4.json', 'r') as fp:
    bc4 = np.array(json.load(fp))

with open(CURRENT_MODEL_SAVE_DIR+pre+'5.json', 'r') as fp:
    bc5 = np.array(json.load(fp))

with open(CURRENT_MODEL_SAVE_DIR+pre+'6.json', 'r') as fp:
    bc6 = np.array(json.load(fp))

#appy transforms
mult = .12
bc = np.stack((bc0,bc1,bc2,bc3,bc4,bc5,bc6), axis=0)*mult

bc0.shape,bc.shape, np.min(bc), np.max(bc)
# timesteps/petals, morphSteps/batch, crossPoints/inputs, coordinates


((7, 7, 3), (7, 7, 7, 3), -0.5523978996276855, 1.0635079193115233)

In [8]:
''' 
2. Convert Blender xyz tensors to NN tensor format

'''
nc0= gy.FormatBlenderToNetwork(bc[0], typeList=[], hasTime=0, hasZ=1, numPoints=7)
nc1= gy.FormatBlenderToNetwork(bc[1], typeList=[], hasTime=0, hasZ=1, numPoints=7)
nc2= gy.FormatBlenderToNetwork(bc[2], typeList=[], hasTime=0, hasZ=1, numPoints=7)
nc3= gy.FormatBlenderToNetwork(bc[3], typeList=[], hasTime=0, hasZ=1, numPoints=7)
nc4= gy.FormatBlenderToNetwork(bc[4], typeList=[], hasTime=0, hasZ=1, numPoints=7)
nc5= gy.FormatBlenderToNetwork(bc[5], typeList=[], hasTime=0, hasZ=1, numPoints=7)
nc6= gy.FormatBlenderToNetwork(bc[6], typeList=[], hasTime=0, hasZ=1, numPoints=7)

nc = np.stack((nc0,nc1,nc2,nc3,nc4,nc5,nc6), axis=0)

bc0.shape, nc0.shape, nc6.shape, nc.shape, np.min(nc), np.max(nc)

((7, 7, 3),
 (7, 21),
 (7, 21),
 (7, 7, 21),
 -0.5523978996276855,
 1.0635079193115233)

In [9]:
nx = np.concatenate((nc[0], nc[1], nc[2], nc[3], nc[4], nc[5], nc[6]))
gx = gy.FormatNetworkToGraphColumns(nx, xOffset=0, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.4,1.3],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.6,1.2],
    )
)
layout.title = 'Blender generated curves XZ Front View'
f = gy.graphCurves(gx, layout, dashRange=range(7,14))
f = gy.graphCurves(gx, layout, dashRange=gy.rangeList([range(7,14),range(21,28)]))
offline.iplot(f)


In [10]:
gx = gy.FormatNetworkToGraphColumns(nc[4], xOffset=7, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.8,.8],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.6,1],
    )
)
layout.title = 'Blender generated curves YZ Right Side View'
f = gy.graphCurves(gx, layout,dashRange=[10,10])
offline.iplot(f)

In [11]:
gx = gy.FormatNetworkToGraphColumns(nc[4], xOffset=0, yOffset=7, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.1,1.2],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.5,.5],
    )
)
layout.title = 'Blender generated curves XY Top View View'
labels = [0.,0.1,.3,.55,.8,1.0, 1.1]
f = gy.graphCurves(gx, layout, labels,dashRange=range(10,100))
# offline.iplot(f)
# if not os.path.exists('images'):
#     os.mkdir('images')
# pio.write_image(f, 'images/fig1.png')


In [13]:
''' 
3. Create reference NN input and output Targets

'''
morphIndices=[0.,0.1,.3,.55,.8,1.0, 1.1]
timeIndices= [0.,0.1,.3,.55,.8,1.0, 1.1]

relMapIn = gy.createRmInput(morphIndices, timeIndices, sPermutation=[])
trelMapIn = torch.Tensor(relMapIn)
tphTarget = torch.Tensor(nc)

trelMapIn.shape, tphTarget.shape


(torch.Size([7, 7, 2]), torch.Size([7, 7, 21]))

In [14]:
'''
4. Initialize network model

'''

torch.manual_seed(1)
templateModel = Net([2,10,15,21])
templateModel.train = True

#device = torch.device("cuda:0")
device = torch.device("cpu")
print('cuda device: ', device)
if device == torch.device("cuda:0"):
    templateModel.to(device)
    trelMapIn = tgeTarget.to(device)
    tphTarget = tphTarget.to(device)
    print("Transfer to use ", device)
    
templateModel.shape, trelMapIn.shape

cuda device:  cpu


([2, 10, 15, 21], torch.Size([7, 7, 2]))

In [23]:
''' 
5. Train 2D Ge input to NN model 3D (xyz) Ph output

'''
numBatch = trelMapIn.shape[0]
criterion = torch.nn.MSELoss(reduction='sum')
start = time.time()

parms={'lRate':.00005, 'epochs':20000,'numBatch':numBatch,'relMap':trelMapIn,'phTarget':tphTarget}
phTrain(templateModel,parms)

print('time: ', time.time()-start)
templateModel.train = False

Epoch 0 - loss: 0.00308
Epoch 2,000 - loss: 0.00382
Epoch 4,000 - loss: 0.00754
Epoch 6,000 - loss: 0.00418
Epoch 8,000 - loss: 0.00912
Epoch 10,000 - loss: 0.00690
Epoch 12,000 - loss: 0.00913
Epoch 14,000 - loss: 0.00757
Epoch 16,000 - loss: 0.00504
Epoch 18,000 - loss: 0.00382
Epoch 20,000 - loss: 0.00122:0.04446
time:  127.56095957756042


In [ ]:
'''
parms={'lRate':.001, 'epochs':20000,'numBatch':numBatch,'relMap':trelMapIn,'phTarget':tphTarget}
Epoch 20,000 - loss: 0.00370:38.28113
Epoch 20,000 - loss: 0.00284:0.27083
parms={'lRate':.0005, 'epochs':20000,'numBatch':numBatch,'relMap':trelMapIn,'phTarget':tphTarget}
Epoch 20,000 - loss: 0.00177:0.09617
Epoch 20,000 - loss: 0.00137:0.04988
parms={'lRate':.0002, 'epochs':20000,'numBatch':numBatch,'relMap':trelMapIn,'phTarget':tphTarget}
Epoch 20,000 - loss: 0.00138:0.05712
parms={'lRate':.00005, 'epochs':20000,'numBatch':numBatch,'relMap':trelMapIn,'phTarget':tphTarget}
Epoch 20,000 - loss: 0.00122:0.04446

'''

In [24]:
'''
6. Generate sample model output and blender formatted test data

'''

morphIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]
timeIndices=[0.,0.1,.2,.3,.5,.7,.9,1.0, 1.1]
relMapIn = gy.createRmInput(morphIndices, timeIndices, sPermutation=[])

ph2t = templateModel(torch.Tensor(relMapIn)).data.numpy()

relMapIn.shape,ph2t.shape

((9, 9, 2), (9, 9, 21))

In [25]:
# model generated
grPh2t = np.reshape(ph2t,(-1,21))
gx = gy.FormatNetworkToGraphColumns(grPh2t, xOffset=0, yOffset=14, colPoints=7)

layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.8,1.3],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-1,1.3],
    )
)
layout.title = 'Model generated curves XZ Front View'
f = gy.graphCurves(gx, layout, dashRange=range(0,8))
offline.iplot(f)

In [26]:
i = 5
ii = 6
xOffset=0
yOffset=7
colPoints=7

gx1 = gy.FormatNetworkToGraphColumns(nc[i], xOffset, yOffset, colPoints)
gx2 = gy.FormatNetworkToGraphColumns(ph2t[ii], xOffset, yOffset, colPoints)
gx = np.concatenate((gx1,gx2),axis=0)
layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.1,1.2],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.6,.6],
    )
)
layout.title = 'Model generated curves XY Top View'
f = gy.graphCurves(gx, layout, dashRange=range(0,7))
offline.iplot(f)

In [27]:
i = 2
ii = 3
xOffset=7
yOffset=14
colPoints=7

gx1 = gy.FormatNetworkToGraphColumns(nc[i], xOffset, yOffset, colPoints)
gx2 = gy.FormatNetworkToGraphColumns(ph2t[ii], xOffset, yOffset, colPoints)
gx = np.concatenate((gx1,gx2),axis=0)
layout = go.Layout(
    title = 'Initial Curves',
    height = 600,
    width = 600,
    hovermode= 'closest',
    xaxis= dict(
        title= 'X',
        range = [-.3,.3],
    ),
    yaxis=dict(
        title= 'Y',
        range = [-.1,1.3],
    )
)
layout.title = 'Model generated curves XY Top View'
f = gy.graphCurves(gx, layout, dashRange=range(0,7))
offline.iplot(f)

In [18]:
'''
7. Save 2D template model weights to file

'''

# PATH = CURRENT_MODEL_SAVE_DIR +'templateModel0.pth'
# torch.save(phModel.state_dict(), PATH)
# PATH

'\n7. Save 2D template model weights to file\n\n'

In [19]:
# bphOut = gy.FormatNetworkToBlender(phOut, xOffset=0, yOffset=7, zOffset=14, colPoints=7)
# PATH = CURRENT_MODEL_SAVE_DIR +'bModelT0.json'

# with open(PATH, 'w') as fp:
#     json.dump(bphOut.tolist(),fp)
    
# PATH, bphOut.shape